In [21]:
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:reporting-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
%use dataframe
%use kandy

In [22]:
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val REPORT_DATA_CONFIGURATION_FILE = "../../data/provision/expenses-report-data-configuration.json"

val userSdk = UserSdk()
val username = "Johann-12.1"
val user = runBlocking {
    userSdk.findUserByUsername(username)
        ?: userSdk.createUser(username)
}

user

UserTO(id=a4894ef4-bb5d-4288-bc0c-a77c2b189cc6, username=Johann-12.1)

In [23]:
import ro.jf.funds.fund.sdk.FundSdk

val fundSdk = FundSdk()
val expenseFund = runBlocking { fundSdk.getFundByName(user.id, "Expenses") }
expenseFund

FundTO(id=c8fab236-25f1-43d0-a8c8-0f5331821c68, name=Expenses)

In [24]:
import ro.jf.funds.reporting.api.model.*
import kotlinx.serialization.json.Json

fun getReportDataConfiguration(): ReportDataConfigurationTO {
    val dataConfigurationRawJson: String = File(REPORT_DATA_CONFIGURATION_FILE).readText()
    val jsonFormat = Json { ignoreUnknownKeys = true }
    val dataConfiguration = jsonFormat.decodeFromString<ReportDataConfigurationTO>(dataConfigurationRawJson)
    return dataConfiguration
}
val dataConfiguration = getReportDataConfiguration()
dataConfiguration

ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=[ReportGroupTO(name=basic, filter=RecordFilterTO(labels=[basic])), ReportGroupTO(name=home, filter=RecordFilterTO(labels=[home])), ReportGroupTO(name=shopping_services, filter=RecordFilterTO(labels=[shopping_services])), ReportGroupTO(name=transport, filter=RecordFilterTO(labels=[transport])), ReportGroupTO(name=fun, filter=RecordFilterTO(labels=[fun])), ReportGroupTO(name=gifts, filter=RecordFilterTO(labels=[gifts])), ReportGroupTO(name=development, filter=RecordFilterTO(labels=[development]))], features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=true)))

In [25]:
import ro.jf.funds.reporting.sdk.ReportingSdk
import kotlinx.coroutines.delay
import ro.jf.funds.commons.model.labelsOf
import ro.jf.funds.commons.model.Currency
import java.time.Instant

val reportingSdk = ReportingSdk()
val reportView = runBlocking {
    val reportViewName = "Expense report 2"
    val existingReportView = reportingSdk.listReportViews(user.id).items.firstOrNull { it.name == reportViewName }
    if (existingReportView != null) {
        return@runBlocking existingReportView
    }
    val request = CreateReportViewTO(reportViewName, expenseFund.id, dataConfiguration)
    var task: ReportViewTaskTO = reportingSdk.createReportView(user.id, request)
    val timeout = Instant.now().plusSeconds(120)
    while (task is ReportViewTaskTO.InProgress && Instant.now().isBefore(timeout)) {
        delay(2000)
        task = reportingSdk.getReportViewTask(user.id, task.taskId)
    }
    if (task is ReportViewTaskTO.Completed) {
        task.report
    } else {
        throw IllegalStateException("Report view creation timed out on task $task")
    }
}
reportView

ReportViewTO(id=40d3ef0b-9c38-41e2-94d7-8e1a2177c1b4, name=Expense report 2, fundId=c8fab236-25f1-43d0-a8c8-0f5331821c68, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=null, features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=false))))

In [26]:
import ro.jf.funds.reporting.api.model.*

val granularInterval = GranularDateInterval(
    interval = DateInterval(
        LocalDate.parse("2019-01-01"),
        LocalDate.parse("2019-12-31")
    ),
    granularity = TimeGranularity.MONTHLY
)
val report = runBlocking {
    reportingSdk.getReportViewData(user.id, reportView.id, granularInterval)
}
report

ReportDataTO(viewId=40d3ef0b-9c38-41e2-94d7-8e1a2177c1b4, granularInterval=GranularDateInterval(interval=DateInterval(from=2019-01-01, to=2019-12-31), granularity=MONTHLY), data=[ReportDataItemTO(timeBucket=DateInterval(from=2019-01-01, to=2019-01-31), net=-4468.51, value=ValueReportTO(start=0.0, end=2481.52, min=0.0, max=0.0), groups=[ReportDataGroupItemTO(group=basic, net=-2305.04), ReportDataGroupItemTO(group=home, net=-500.0), ReportDataGroupItemTO(group=shopping_services, net=-385.57), ReportDataGroupItemTO(group=transport, net=-204.24), ReportDataGroupItemTO(group=fun, net=-1031.16), ReportDataGroupItemTO(group=gifts, net=0.0), ReportDataGroupItemTO(group=development, net=-42.5)]), ReportDataItemTO(timeBucket=DateInterval(from=2019-02-01, to=2019-02-28), net=-4301.32, value=ValueReportTO(start=2481.52, end=5185.08, min=0.0, max=0.0), groups=[ReportDataGroupItemTO(group=basic, net=-2509.31), ReportDataGroupItemTO(group=home, net=-609.16), ReportDataGroupItemTO(group=shopping_servi

In [27]:
import java.math.BigDecimal
import org.jetbrains.kotlinx.kandy.dsl.* // For creating visualizations
import org.jetbrains.letsPlot.scale.scaleYContinuous

data class MonthlyExpense(val month: LocalDate, val net: Double)

val monthlyReportDF = report.data
    .mapNotNull { reportDataItem ->
        reportDataItem.net?.let { amount ->
        MonthlyExpense(reportDataItem.timeBucket.from, amount.negate().toDouble())
            }
    }
    .toDataFrame()

plot(monthlyReportDF) {
    y("net")
    bars {
        x("month") {

        }
        fillColor = Color.ORANGE
    }
    layout {
        title = "Monthly expenses"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="dI47pU"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Monthly expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12],
"net":[4468.51,4301.32,9171.45,6967.76,9043.57,7024.09,11232.58,25034.218092,4537.4484992,6776.5,9787.68,5529.22]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"net",
"x":"month"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"fill":"#fc8452",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"column":"month",
"type":"datetime"
}]
}
};
 var plotContainer = document.getElementById("dI47pU");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Jan 
 
 
 
 
 
 
 
 
 Feb 
 
 
 
 
 
 
 
 
 Mar 
 
 
 
 
 
 
 
 
 Apr 
 
 
 
 
 
 
 
 
 May 
 
 
 
 
 
 
 
 
 Jun 
 
 
 
 
 
 
 
 
 Jul 
 
 
 
 
 
 
 
 
 Aug 
 
 
 
 
 
 
 
 
 Sep 
 
 
 
 
 
 
 
 
 Oct 
 
 
 
 
 
 
 
 
 Nov 
 
 
 
 
 
 
 
 
 Dec 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 6,000 
 
 
 
 
 
 
 8,000 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 12,000 
 
 
 
 
 
 
 14,000 
 
 
 
 
 
 
 16,000 
 
 
 
 
 
 
 18,000 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 22,000 
 
 
 
 
 
 
 24,000 
 
 
 
 
 
 
 26,000 
 
 
 
 
 
 
 
 
 Monthly expenses 
 
 
 
 
 net 
 
 
 
 
 month

In [38]:
val categories = mapOf(
    "basic" to Color.GREEN,
    "home" to Color.PEACH,
    "shopping_services" to Color.LIGHT_GREEN,
    "transport" to Color.BLUE,
    "fun" to Color.ORANGE,
    "gifts" to Color.RED,
    "development" to Color.PURPLE,
)

fun getGroupedExpensesDataFrame(): DataFrame<*> {
    val categoriesNetExtractor: Map<String, (List<ReportDataGroupItemTO>) -> Double> =
        categories
            .keys
            .map { group ->
                group to { groups: List<ReportDataGroupItemTO> ->
                    groups.first { it.group == group }.net?.negate()?.toDouble() ?: 0.0
                }
            }
            .toMap()

    return report.data
        .map { monthlyData ->
            monthlyData.timeBucket.from to categoriesNetExtractor.mapValues { (_, extractor) -> extractor(monthlyData.groups!!) }
        }
        .let {
            dataFrameOf(
                "month" to it.flatMap { (month, values) -> values.map { month } },
                "value" to it.flatMap { (_, values) -> values.map { it.value } },
                "category" to it.flatMap { (_, values) -> values.keys }
            )
        }
}
getGroupedExpensesDataFrame().groupBy("category").plot {
    x("month")
    line {
        y("value")
        color("category") {
            scale = categorical(
                *categories.map { (category, color) -> category to color }.toTypedArray()
            )
        }
    }
    line {
        y.constant(0)
    }
    layout {
        title = "Expenses by category"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="XdAcwU"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Expenses by category"
},
"mapping":{
},
"data":{
"&merged_groups":["basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","home","home","home","home","home","home","home","home","home","home","home","home","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","development","development","development","development","development","development","development","development","development","development","development","development"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12],
"category":["basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","home","home","home","home","home","home","home","home","home","home","home","home","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","development","development","development","development","development","development","development","development","development","development","development","development"],
"value":[2305.04,2509.31,3347.92,2293.91,2108.18,2382.69,3709.61,2493.78,1587.87,2253.23,2992.86,2517.62,500.0,609.16,1763.9,400.0,400.0,467.23,420.0,18881.0,423.24,456.78,400.0,400.0,385.57,0.0,609.77,1481.1,936.99,2494.99,463.18,0.0,82.99,342.57,0.0,0.0,204.24,207.85,1214.12,281.75,554.64,348.18,288.04,1474.32,331.47,238.42,2502.33,2020.41,1031.16,570.0,1237.89,2109.0,2235.76,0.0,790.09,1980.318092,1753.8784992,405.5,0.0,100.0,0.0,392.0,147.85,402.0,1930.0,1330.0,5561.66,204.8,351.0,20.0,3892.49,491.19,42.5,13.0,850.0,0.0,878.0,1.0,0.0,0.0,7.0,3060.0,0.0,0.0]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"pl

In [ ]:
data class MonthlyExpense(val month: LocalDate, val start: Double, val end: Double)

val df = report.data
    .mapNotNull { item ->
        item.value?.let { valueReport ->
            MonthlyExpense(
                item.timeBucket.from,
                valueReport.start.toDouble(),
                valueReport.end.toDouble()
            )
        }
    }
    .let {
        dataFrameOf(
            "month" to it.map { it.month }.let { it + it },
            "value" to it.map { it.start } + it.map { it.end },
            "type" to List(it.size) { "startValue" } + List(it.size) { "endValue" }
        )
    }

df.groupBy("type").plot {
    x("month")
    line {
        y("value")
        color("type") {
            scale = categorical("startValue" to Color.ORANGE, "endValue" to Color.BLUE)
        }
    }
    line {
        y.constant(0)
    }
    layout {
        title = "Expense fund value"
        size = 1200 to 600
    }
}